# Introduction

Introduction cell, the introduction will consist of:

- Introduction text
- Overview of previous module content
- Table of contents for this module

# What is a CSV File?

# Reading a CSV File

# Removing Implausible Values

# Dealing with Missing Values

# Module Conclusion

Concluding cell, the conclusion will consist of

- Brief overview of module content
- Connection to end of module quiz